In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
driver = webdriver.Chrome("/home/bernardo/Downloads/chromedriver")
driver2 = webdriver.Chrome("/home/bernardo/Downloads/chromedriver")

In [18]:
driver.get("https://codes.findlaw.com/LCsearch.html#?cludoquery=census%20population&cludoCategory=Codes&cludoRegion=Michigan&cludopage=1&cludorefurl=https%3A%2F%2Fcodes.findlaw.com%2Fmi%2F&cludorefpt=Michigan%20Compiled%20Laws%20%7C%20FindLaw")

In [ ]:
codes = []
page = 1
content = driver.page_source
soup = BeautifulSoup(content)
next_len = len(soup.findAll("a",attrs={"data-page":f"{page+1}"}))

while page==1 or next_len>0:
    if page>1:
        right_arrow =  driver.find_element(By.CSS_SELECTOR,f"a[data-page='{page}']").click()
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.search-results-item"))
            )
        finally:
            content = driver.page_source
            soup = BeautifulSoup(content)
            next_len = len(soup.findAll("a",attrs={"data-page":f"{page+1}"}))
        
    for a in soup.findAll('li',attrs={"class":"search-results-item"}):
        b= a.findNext('a',href=True)
        
        driver2.get(b["href"])
        try:
            element = WebDriverWait(driver2, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='search']"))
            )
        finally:
            content2 = driver2.page_source
            soup2 = BeautifulSoup(content2)
            text =[]

            if "404 Error" not in driver2.title:
                div_content = soup2.findAll('div',attrs={"class":"leafContent section"})
                if len(div_content)>0:
                    c =div_content[0].findAll("div",attrs={'class':'subsection'})
                    for d in c:
                        text.append(d.get_text())

            text = "\n".join(text)
            codes.append({
                "title": b["title"],
                "href": b["href"],
                "text": text
            })
   
        
        
    
    page=page+1
    

In [10]:
df = pd.DataFrame.from_records(codes)
df

,title,href,text
0,Connecticut General Statutes Title 10. Educati...,https://codes.findlaw.com/ct/title-10-educatio...,\n(a) Whenever used in this section and \nsect...
1,Connecticut General Statutes Title 12. Taxatio...,https://codes.findlaw.com/ct/title-12-taxation...,
2,Connecticut General Statutes Title 32. Commerc...,https://codes.findlaw.com/ct/title-32-commerce...,\n(a) Any municipality that was a distressed m...
3,Connecticut General Statutes Title 7. Municipa...,https://codes.findlaw.com/ct/title-7-municipal...,\n(a) As used in this section:\n\n(1) “Secreta...
4,Connecticut General Statutes Title 30. Intoxic...,https://codes.findlaw.com/ct/title-30-intoxica...,\n(a) A hotel permit shall allow the retail sa...
...,...,...,...
388,Connecticut General Statutes Title 32. Commerc...,https://codes.findlaw.com/ct/title-32-commerce...,\n(a) As used in this section:\n\n(1) “Commiss...
389,Connecticut General Statutes Title 36A. The Ba...,https://codes.findlaw.com/ct/title-36a-the-ban...,\n(a) One or more persons may organize a Conne...
390,Connecticut General Statutes Title 54. Crimina...,https://codes.findlaw.com/ct/title-54-criminal...,ARTICLE IPURPOSEThe compacting states to this ...
391,Connecticut General Statutes Title 32. Commerc...,https://codes.findlaw.com/ct/title-32-commerce...,"\n(a) The commissioner shall, within available..."


In [11]:
df.to_csv("data/Michigan.csv",index=False)